<a href="https://colab.research.google.com/github/ak1920/ML/blob/master/Copy_of_01_Introduction_to_Scikit_learn_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Introduction to Scikit-learn

### Step 1. Import the necessary libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### Step 2. Import one of the toy datasets (digits) from Scikitlearn
This is a copy of the test set of the UCI ML hand-written digits datasets https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where each class refers to a digit.

Each datapoint is a 8x8 image of a digit, Classes=10, Samples per class~180, 

Samples total=1797, Dimensionality=64, and Features=(integers 0-16)


After importing this dataset, split it into test and train sets. You may check the shape of data and the target attributes of the dataset. You may also want to print a few samples from the dataset.

In [3]:
from sklearn.datasets import load_digits
digits = load_digits()

In [4]:
digits.keys()

dict_keys(['data', 'target', 'target_names', 'images', 'DESCR'])

In [5]:
X = digits.data
y = digits.target

X.shape

(1797, 64)

In [6]:
y[30:40]

array([0, 9, 5, 5, 6, 5, 0, 9, 8, 9])

In [7]:
np.bincount(y)

array([178, 182, 177, 183, 181, 182, 181, 179, 174, 180])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

In [11]:
np.bincount(y_train)

array([141, 139, 133, 138, 143, 134, 129, 131, 126, 133])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=0)

In [14]:
np.bincount(y_train)

array([133, 136, 133, 137, 136, 136, 136, 134, 131, 135])

In [15]:
X_train[0,:]

array([ 0.,  0.,  0., 10., 12., 15., 16., 13.,  0.,  0.,  6., 15.,  6.,
        4., 14.,  9.,  0.,  0., 10.,  6.,  0.,  3., 14.,  2.,  0.,  1.,
       14.,  1.,  0., 12.,  6.,  0.,  0.,  0.,  3.,  0.,  5., 13.,  0.,
        0.,  0.,  0.,  0.,  1., 13.,  3.,  0.,  0.,  0.,  0.,  0.,  6.,
       13.,  0.,  0.,  0.,  0.,  0.,  0., 14.,  6.,  0.,  0.,  0.])

In [16]:
y_train[0]

7

# II. Sklearn API for model training
-------------------
### Step 1. Import your model class

As an example, let us `LinearSVC`, a linear support vector classifier. This classifier is imprted from `sklearn.svm` module which includes Support Vector Machine algorithms.

In [18]:
from sklearn.svm import LinearSVC


### Step2. Instantiate an object and set the parameters

In [19]:
svm = LinearSVC()

### Step 3. Fit the model
When fitting the model, use the train dataset.

In [20]:
svm.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

### Step 4. Predict and Evaluate
Use the test set for this purpose, for now.

In [22]:
print(svm.predict(X_train[:10,:]))
print(y_train[:10])

[7 3 6 6 7 6 7 9 2 9]
[7 3 6 6 7 6 7 9 2 9]


In [23]:
svm.score(X_train, y_train)

0.9925760950259837

In [24]:
svm.score(X_test, y_test)

0.94

### Step5. Try another Algorithm
Try `RandomForestCLassifier` this time, import it from `sklearn.ensemble` module.

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rf = RandomForestClassifier(n_estimators=50)

In [27]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
print(rf.score(X_train, y_train))
print(rf.score(X_test, y_test))

1.0
0.9733333333333334


# II. Cross-validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set X_test, y_test. 

When evaluating different settings (“hyperparameters”) for estimators, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

A model is trained using  of the folds as training data; the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.
(Check https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [30]:
scores = cross_val_score(KNeighborsClassifier(), X_train, y_train, cv=5)
print(scores)
    

[0.98148148 0.97777778 0.98513011 0.98513011 0.98513011]


In [34]:
print("Accuracy of Model: {:2f}(+/-{:.2f})". format(scores.mean(), scores.std()*2))

Accuracy of Model: 0.982930(+/-0.01)


In [35]:
from sklearn.model_selection import KFold, StratifiedKFold

In [36]:
kfold= KFold(n_splits=10, shuffle=True, random_state=10)
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

In [40]:
scores_kfold = cross_val_score(KNeighborsClassifier(), X_train, y_train, cv=kfold)
scores_skfold = cross_val_score(KNeighborsClassifier(), X_train, y_train, cv=skfold)

In [41]:
print("Accuracy of Model with kfold: {:2f}(+/-{:.2f})". format(scores_kfold.mean(), scores_kfold.std()*2))

Accuracy of Model with kfold: 0.985904(+/-0.02)


In [42]:
print("Accuracy of Model with skfold: {:2f}(+/-{:.2f})". format(scores_skfold.mean(), scores_skfold.std()*2))

Accuracy of Model with skfold: 0.985158(+/-0.02)


Grid Searches
=================
Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

See (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Grid-Search with build-in cross validation

A GridSearchCV object behaves just like a normal classifier.

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [46]:
import numpy as np

In [47]:
10.**np.arange(-3,3)

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [50]:
param_grid = {'C': 10.**np.arange(-3,3),
              'gamma': 10.**np.arange(-5,0)}

In [51]:
np.logspace(-3,3,6, endpoint=False)

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [49]:
print(param_grid)

{'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'gamma': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])}


In [53]:
grid_search= GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)

In [54]:
grid_search

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'gamma': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [55]:
mygrid = grid_search.fit(X_train, y_train)

In [58]:
import pandas as pd


In [56]:
GridResultdf.columns

NameError: ignored

In [ ]:
predictions = grid_search.predict(X_test)

In [ ]:
print("{:.2f}".format(grid_search.score(X_test, y_test)))

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [57]:
scores = grid_search.cv_results_('mean_test_score')

TypeError: ignored

In [ ]:
plt.matshow(scores, cmap = plt.cm.BuPu_r)
plt.colorbar()
plt.show()